# Final Project: [HighwayEnv](https://github.com/Farama-Foundation/HighwayEnv/tree/master)

Ressources:
- **Highway-env** [👨‍💻Repo](https://github.com/Farama-Foundation/HighwayEnv/tree/master) | [📜Documentation](http://highway-env.farama.org/quickstart/)
- **OpenAI Gym**
- **Stable-Baselines3**: [👨‍💻Repo](https://github.com/DLR-RM/stable-baselines3) | [📜Documentation](https://stable-baselines.readthedocs.io/en/master/)

### Your task: Solve the Highway
![](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/highway.gif?raw=true)
- By Group of two
- Use *at least* two different RL Algorithms
  - try to implement at least one 'by hand'

### Evaluation
*Based on the report (showing that you understood what you did), the performances and the code (you did something that works).*

- **Produce a notebook**
  -  The notebook must run one one go, I will not loose time trying to fix your env...
  - Possible to send a git repo with the weight so that I ca nrun them locally.
- **Produce a 2-5 pages report**
  - Describe Your choices and explain the algorithms used.
  - Benchmark and compare them depending on their hyperparameters.

*Analysis could include exploration of hyperparameters, figures of training, explainations of how your algorithm works*

### Roadmap
- 📆 **04 janv** : Send a report (5-10 pages) and a notebook / script



## Utlilities
⚠️ *Do not Modify anything here !*

but always read everything to be sure of what is available

### Imports

In [7]:
!pip install gymnasium>=1.0.0a2
!pip install farama-notifications>=0.0.1
!pip install numpy>=1.21.0
!pip install pygame>=2.0.2
!pip install stable-baselines3[extra]
!pip install highway_env
#tensorboard loading if you want to use it
%load_ext tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 14.1 MB/s eta 0:00:00


### Utils

In [8]:
### VIDEO RECORDER
# Set up fake display; otherwise rendering will fail
import os
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from tqdm import tqdm

os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/", fps = 10):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    eval_env.metadata["render_fps"] = fps
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )
    obs = eval_env.reset()
    for _ in tqdm(range(video_length)):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 200px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [9]:
# prompt: define an evaluation function computing mean reward and elapsed episode time on a few runs of vectorized environments
import numpy as np

def evaluate(model, num_episodes=30):
    """
    Evaluates a reinforcement learning agent.

    Args:
        model: The trained RL model.
        env: The environment to evaluate the model on.
        num_episodes: The number of episodes to run for evaluation.

    Returns:
        A tuple containing the mean reward and the mean elapsed time per episode.
    """
    env_id = "racetrack-v0"
    env = make_vec_env(env_id)
    episode_rewards = []
    episode_times = []
    print(f"evaluating Model on {num_episodes} episodes ...")
    for _ in tqdm(range(num_episodes)):
        obs = env.reset()
        done = False
        total_reward = 0
        start_time = 0 # Assuming env provides time information. Replace with actual time tracking
        current_time = 0

        while not done:
          action, _states = model.predict(obs, deterministic=True)
          obs, reward, done, info = env.step(action)
          total_reward += reward
          current_time += 1 # Replace with actual elapsed time from env info

        episode_rewards.append(total_reward)
        episode_times.append(current_time - start_time)

    mean_reward = np.mean(episode_rewards)
    mean_time = np.mean(episode_times)
    std_reward = np.std(episode_rewards)
    std_time = np.std(episode_times)
    print(f"\n{'-'*50}\nResults :\n\t- Mean Reward: {mean_reward:.3f} ± {std_reward:.2f} \n\t- Mean elapsed Time per episode: {mean_time:.3f} ± {std_time:.2f}\n{'-'*50}")
    return mean_reward, mean_time


## The Highway Environment

In [10]:
## IMPORTS
import gymnasium as gym
from stable_baselines3 import PPO, DQN, SAC, A2C, TD3
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import highway_env  # noqa: F401

/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

## Load and explore Environment
Lets first load an untrained model and see how it behaves in the environment.

In [11]:
env_id = "racetrack-v0"
env = gym.make(env_id)

#instanciate model
model = PPO("MlpPolicy", env, verbose=1)

#generate video of random model
record_video(env_id, model, video_length=50, prefix="random-agent", fps = 5)
show_videos("videos", prefix="random-agent")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
 98%|█████████▊| 49/50 [00:04<00:00,  7.97it/s]

Saving video to /content/videos/random-agent-step-0-to-step-50.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
 98%|█████████▊| 49/50 [00:05<00:00,  7.97it/s]

Moviepy - Building video /content/videos/random-agent-step-0-to-step-50.mp4.
Moviepy - Writing video /content/videos/random-agent-step-0-to-step-50.mp4




100%|██████████| 50/50 [00:05<00:00,  8.43it/s]

Moviepy - Done !
Moviepy - video ready /content/videos/random-agent-step-0-to-step-50.mp4


In [12]:
evaluate(model)

evaluating Model on 30 episodes ...


100%|██████████| 30/30 [00:15<00:00,  1.99it/s]


--------------------------------------------------
Results :
	- Mean Reward: 16.902 ± 5.01 
	- Mean elapsed Time per episode: 20.300 ± 5.47
--------------------------------------------------


(np.float32(16.902205), np.float64(20.3))

Let's now explore the environments settings:
### Action Space
👉 Look at the action space, what actions can the model do ?

In [13]:
print(f"   Type: {type(env.action_space)}")
print(f"   Details: {env.action_space}")

   Type: <class 'gymnasium.spaces.box.Box'>
   Details: Box(-1.0, 1.0, (1,), float32)


In [14]:
if hasattr(env.action_space, 'n'):
    print(f"   Number of discrete actions: {env.action_space.n}")
    print(f"   Actions are: 0 to {env.action_space.n - 1}")
elif hasattr(env.action_space, 'shape'):
    print(f"   Continuous action shape: {env.action_space.shape}")
    print(f"   Action bounds: low={env.action_space.low}, high={env.action_space.high}")

   Continuous action shape: (1,)
   Action bounds: low=[-1.], high=[1.]


### Observation Space
👉 Look at the [documentation](http://highway-env.farama.org/observations/) for possibles observations of the agents on the Highway

👉 Look at the observation spae in our case

In [15]:
print(f"   Type: {type(env.observation_space)}")
print(f"   Shape: {env.observation_space.shape}")
print(f"   Details: {env.observation_space}")


   Type: <class 'gymnasium.spaces.box.Box'>
   Shape: (2, 12, 12)
   Details: Box(-inf, inf, (2, 12, 12), float32)


In [16]:
print("\n SAMPLE OBSERVATION:")
obs, info = env.reset()
print(f"   Observation shape: {obs.shape}")
print(f"   Observation type: {type(obs)}")
print(f"   Sample observation (first few values):")
print(obs[:3] if len(obs.shape) > 1 else obs[:5])


 SAMPLE OBSERVATION:
   Observation shape: (2, 12, 12)
   Observation type: <class 'numpy.ndarray'>
   Sample observation (first few values):
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
  [1. 0. 1. 0. 

In [17]:
# 4. EPISODE LENGTH & REWARD
print("\n RUNNING SAMPLE EPISODE (Random Actions):")
obs, info = env.reset()
total_reward = 0
steps = 0
done = False

while not done and steps < 100:  # Max 100 steps for testing
    action = env.action_space.sample()  # Random action
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    total_reward += reward
    steps += 1

print(f"   Episode length: {steps} steps")
print(f"   Total reward: {total_reward:.2f}")
print(f"   Average reward per step: {total_reward/steps:.3f}")
print(f"   Episode ended with: {'terminated' if terminated else 'truncated'}")

env.close()



 RUNNING SAMPLE EPISODE (Random Actions):
   Episode length: 9 steps
   Total reward: 4.79
   Average reward per step: 0.532
   Episode ended with: terminated


# Training an Agent on the Environment
👉 **Now it is your turn**, train your agents
Recall:
- you must try and compare different RL Algorithms
- part of your grade will be the evaluation of your best Agent.

🔥Tips
- Use tensorboard to monitor your trainings
- install it locally to get faster and longer trainings (not mandatory, colab should be ok)

In [ ]:
# if you want to use tensorboard, highly recommended
%tensorboard --logdir "highway"

In [ ]:
######### YOUR CODE HERE #########


In [ ]:
######### SOME OTHER FANCY TRAINING HERE #########

In [ ]:
### SAVE YOUR FINAL MODEL
model_final = .... #YOUR MODEL
model_final.save("highway_final")

# Evalutation
⚠️ *Do not Modify anything here !*

Now that your Agents are trained, we evaluate them

In [ ]:
evaluate(model_final)

In [ ]:
env_id = "highway-v0"
# Generate video of trained model
record_video(env_id, model_final, video_length=70, prefix="trained-agent", fps = 5)
show_videos("videos", prefix="trained-agent")